In [99]:
import os
import json
import random

root_dir = '../MATH'  # Replace with your directory
num_files_to_load = 1000  # Replace with number of files you want to pick

# Set a seed for randomness
random.seed(123)

json_files = []

# Search for json files in sub-folders of root directory
for dirpath, _, filenames in os.walk(root_dir):
    for f in filenames:
        if f.endswith('.json'):
            json_files.append(os.path.join(dirpath, f))

# Randomly select files
files_to_load = random.sample(json_files, num_files_to_load)

data = []

# Load content from files
for filename in files_to_load:
    with open(filename, 'r') as f:
        data.append(json.load(f))

# print(data)

In [100]:
extracted_data = []
converted_data = []

for datum in data:
    problem = datum.get('problem')
    solution = datum.get('solution')
    # Extract the boxed solution
    boxed_solution = ""
    if "\\boxed{" in solution:
        start = solution.index("\\boxed{") + len("$\boxed{")
        end = solution.index("}", start)
        boxed_solution = solution[start:end]
    extracted_data.append({'problem': problem, 'boxed_solution': boxed_solution})
    
# Check if only numbers and periods in boxed_solution, convert to float
for item in extracted_data:
    boxed_solution = item['boxed_solution']
    if all(c.isdigit() or c == '.' for c in boxed_solution):
        item['boxed_solution'] = float(boxed_solution)
        converted_data.append(item)

# print(converted_data)

In [101]:
with open('sanitized_data.jsonl', 'w') as outfile:
    for entry in converted_data:
        json.dump(entry, outfile)
        outfile.write('\n')

In [7]:
import json
import random
from typing import Any, Dict

def convert_jsonl_file(input_path: str, output_path: str) -> None:
    with open(input_path, 'r') as fr, open(output_path, 'w') as fw:
        i = 0
        for line in fr:
            i += 1
            old_line = json.loads(line)
            new_line = transform_line(old_line, i)
            fw.write(json.dumps(new_line))
            fw.write('\n')

def transform_line(line: Dict[str, Any], i) -> Dict[str, Any]:

    human_eval_format_prompt = f'''
def solveMathProblem():
    """
    Finds the solution to the following math word problem. Returns a float:
    {line["problem"]}
    """
'''

    canonical_solution = f'''
    return {line["boxed_solution"]}
'''

    test_answer = f'''
def check(candidate):
    assert type(candidate) == float, "The answer must be a float."
    min = float({line["boxed_solution"]}) - 0.1
    max = float({line["boxed_solution"]}) + 0.1
    assert min <= candidate <= max, "The answer must be between the min and max values."
'''

    new_line = {
        "task_id": f"MATH_{str(i)}",
        "prompt": human_eval_format_prompt,
        "entry_point": "solveMathProblem",
        "canonical_solution": canonical_solution,
        "test": test_answer
    }
    return new_line

# Use the function to convert the file
convert_jsonl_file('sanitized_data.jsonl', 'output.jsonl')

In [3]:
{"task_id": "HumanEval/114", "prompt": "\ndef minSubArraySum(nums):\n    \"\"\"\n    Given an array of integers nums, find the minimum sum of any non-empty sub-array\n    of nums.\n    Example\n    minSubArraySum([2, 3, 4, 1, 2, 4]) == 1\n    minSubArraySum([-1, -2, -3]) == -6\n    \"\"\"\n", "entry_point": "minSubArraySum", "canonical_solution": "    max_sum = 0\n    s = 0\n    for num in nums:\n        s += -num\n        if (s < 0):\n            s = 0\n        max_sum = max(s, max_sum)\n    if max_sum == 0:\n        max_sum = max(-i for i in nums)\n    min_sum = -max_sum\n    return min_sum\n", "test": "def check(candidate):\n\n    # Check some simple cases\n    assert candidate([2, 3, 4, 1, 2, 4]) == 1, \"This prints if this assert fails 1 (good for debugging!)\"\n    assert candidate([-1, -2, -3]) == -6\n    assert candidate([-1, -2, -3, 2, -10]) == -14\n    assert candidate([-9999999999999999]) == -9999999999999999\n    assert candidate([0, 10, 20, 1000000]) == 0\n    assert candidate([-1, -2, -3, 10, -5]) == -6\n    assert candidate([100, -1, -2, -3, 10, -5]) == -6\n    assert candidate([10, 11, 13, 8, 3, 4]) == 3\n    assert candidate([100, -33, 32, -1, 0, -2]) == -33\n\n    # Check some edge cases that are easy to work out by hand.\n    assert candidate([-10]) == -10, \"This prints if this assert fails 2 (also good for debugging!)\"\n    assert candidate([7]) == 7\n    assert candidate([1, -1]) == -1\n"}

{'task_id': 'HumanEval/114',
 'prompt': '\ndef minSubArraySum(nums):\n    """\n    Given an array of integers nums, find the minimum sum of any non-empty sub-array\n    of nums.\n    Example\n    minSubArraySum([2, 3, 4, 1, 2, 4]) == 1\n    minSubArraySum([-1, -2, -3]) == -6\n    """\n',
 'entry_point': 'minSubArraySum',
 'canonical_solution': '    max_sum = 0\n    s = 0\n    for num in nums:\n        s += -num\n        if (s < 0):\n            s = 0\n        max_sum = max(s, max_sum)\n    if max_sum == 0:\n        max_sum = max(-i for i in nums)\n    min_sum = -max_sum\n    return min_sum\n',
 'test': 'def check(candidate):\n\n    # Check some simple cases\n    assert candidate([2, 3, 4, 1, 2, 4]) == 1, "This prints if this assert fails 1 (good for debugging!)"\n    assert candidate([-1, -2, -3]) == -6\n    assert candidate([-1, -2, -3, 2, -10]) == -14\n    assert candidate([-9999999999999999]) == -9999999999999999\n    assert candidate([0, 10, 20, 1000000]) == 0\n    assert candidate

In [8]:
{"task_id": "MATH_1", "prompt": "\ndef solveMathProblem():\n    \"\"\"\n    Finds the solution to the following math word problem. Returns a float:\n    Abby, Bart, Cindy and Damon weigh themselves in pairs. Together Abby and Bart weigh 160 pounds, Bart and Cindy weigh 180 pounds, and Cindy and Damon weigh 200 pounds. How many pounds do Abby and Damon weigh together?\n    \"\"\"\n", "entry_point": "solveMathProblem", "canonical_solution": "\n    return 180.0\n", "test": "\ndef check(candidate):\n    assert type(candidate) == float, \"The answer must be a float.\"\n    min = float(180.0) - 0.1\n    max = float(180.0) + 0.1\n    assert min <= candidate <= max, \"The answer must be between the min and max values.\"\n"}

{'task_id': 'MATH_1',
 'prompt': '\ndef solveMathProblem():\n    """\n    Finds the solution to the following math word problem. Returns a float:\n    Abby, Bart, Cindy and Damon weigh themselves in pairs. Together Abby and Bart weigh 160 pounds, Bart and Cindy weigh 180 pounds, and Cindy and Damon weigh 200 pounds. How many pounds do Abby and Damon weigh together?\n    """\n',
 'entry_point': 'solveMathProblem',
 'canonical_solution': '\n    return 180.0\n',
 'test': '\ndef check(candidate):\n    assert type(candidate) == float, "The answer must be a float."\n    min = float(180.0) - 0.1\n    max = float(180.0) + 0.1\n    assert min <= candidate <= max, "The answer must be between the min and max values."\n'}